In [1]:

from torch import nn
import torch
import torch.nn.functional as F
import sys

sys.path.append('..')

class TimeSeriesPredictionModel(nn.Module):
    
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True, num_layers=2)
        self.fc = nn.Linear(hidden_size, 36)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x



# Generate sample data for time series prediction (10 samples, each with a sequence length of 30)
sample_x = torch.rand(2, 49, 36)  # 2 samples, each being a sequence of 49 time steps, 36 features
sample_y = torch.rand(2, 49, 36)  # 2 samples, each being a sequence of 49 time steps, 36 features

target_y = torch.zeros_like(sample_y)
# Use an LSTM-based model for time series prediction

model = TimeSeriesPredictionModel(input_size=36, hidden_size=64)

sample_x, sample_y,target_y,model = sample_x.cuda(), sample_y.cuda(),target_y.cuda(),model.cuda()

In [ ]:
# Let's train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for i in range(100):
    optimizer.zero_grad()
    pred_y = model(sample_x)
    loss = criterion(pred_y, sample_y)
    loss.backward()
    optimizer.step()
# Let's test the model
pred_y = model(sample_x)
print(F.mse_loss(pred_y, sample_y))  # This should be close to 0

In [3]:
# before peforimg any attack, We need to define the loss function
# loss function takes only one argument, which is the batch of data
# We can define the loss function as follows
def loss_fn(batch):
    x, y = batch
    output = model(x)
    return F.mse_loss(output, y)

# Or we can define the loss function like this
def loss_fn(model,batch):
    x, y = batch
    output = model(x)
    return F.mse_loss(output, y)

from functools import partial
loss_fn = partial(loss_fn, model) # now loss_fn takes only one argument

#### Attack

In [4]:
from abe.algorithm.attack import FGSM, PGD, BIM, MIFGSM, TIFGSM, DIFGSM, SINIFGSM, SSA

In [5]:
# first we need to define the attack task
from abe.task import AttackTask
from abe.type import ModelType

task = AttackTask(loss_fn=loss_fn, model_type=ModelType.IMAGECLASSIFICATION, is_targeted=False) # for targeted attack, set is_targeted=True

# then we can define the attack algorithm

attack = BIM(task,eps=1,alpha=0.1) # BIM attack

adversarial_x = attack([sample_x, sample_y]) # all attack algorithms take a batch of data as input

targeted_task = AttackTask(loss_fn=loss_fn, model_type=ModelType.IMAGECLASSIFICATION, is_targeted=True)

targeted_attack = BIM(targeted_task,eps=1,alpha=0.1)

targeted_adversarial_x = targeted_attack([sample_x, target_y]) # for targeted attack, the second element of the batch should be the target label

In [ ]:
print("Original prediction: ", model(sample_x).squeeze())
print("Adversarial prediction: ", model(adversarial_x).squeeze())
print("Targeted adversarial prediction: ", model(targeted_adversarial_x).squeeze())

#### Explanation

In [7]:
from abe.algorithm.explanation import AMPE,IG, FastIG, SaliencyMap, SmoothGradient, MFABA

In [8]:
# first we need to define the explanation task
from abe.task import ExplanationTask

# Explanation task takes loss function and forward function as initialization arguments

def loss_fn(batch):
    x, y = batch
    output = model(x)
    return F.mse_loss(output, y)

def forward(batch):
    x, _ = batch
    return model(x)

In [9]:
explanation_task = ExplanationTask(loss_fn=loss_fn, forward_fn=forward, model_type=ModelType.TIMESERIESPREDICTION)

In [10]:
# then we can define the explanation algorithm

explanation = MFABA(explanation_task)

attribution = explanation([sample_x, sample_y])

In [ ]:
attribution.shape

In [ ]:
import seaborn as sns
min_ = attribution[0].min()
max_ = attribution[0].max()
attribution[0] = (attribution[0] - min_) / (max_ - min_)
sns.heatmap(attribution[0].mean(0).reshape(6,6),annot=True,fmt=".2f")
